- RNN(Recurrent Netual Network)

In [1]:
# 데이터 로드 : 전처리 완료
# https://drive.google.com/file/d/1MaRptad6gke9_Y_t4b0AGStHMcc_JGfz/view?usp=sharing
import gdown
file_id = '1MaRptad6gke9_Y_t4b0AGStHMcc_JGfz'
download_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(download_url, 'train_label.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1MaRptad6gke9_Y_t4b0AGStHMcc_JGfz
To: /content/train_label.zip
100%|██████████| 5.47M/5.47M [00:00<00:00, 169MB/s]


'train_label.zip'

In [2]:
!unzip /content/train_label.zip

Archive:  /content/train_label.zip
  inflating: train_input.npy         
  inflating: train_label.npy         


In [3]:
import pandas as pd
import numpy as np

In [4]:
train_data = np.load('train_input.npy')
train_label = np.load('train_label.npy')

In [5]:
train_data.shape, train_label.shape

((25000, 174), (25000,))

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(train_data, train_label, test_size=0.2, random_state=42)

- model
  - tf.estimator.Estimator API를 사용해서 신경망정의
  - 감성분석처럼 이진분류에 사용 LSTM을 포함해서 순환신경망(RNN)을 기반으로

- LSTM(Long Short Term Memory)
  - RNN의 한 종류, 긴 시퀀스에서 중요한 정보를 잃지 않고 기억할수 있도록 설계된 메모리 셀 구조
  - 셀 상태와
    - 장기정보
  - 게이트 상태
    - 단기정보
  - 구조
    - 3가지 게이트
      - Forget Gate : 이전 셀의 상태를 보고 어떤 정보를 버릴지 결정
      - Input Gate : 현재 입력정보를 가지고 새로 저장할 정보
      - Output Gate : 다음단계로 전달할 정보
    - 동작
      - Forget Gate : 시그모이드함수 0에 가까운면 정보를 버리고 1에 가까우면 기억
      - Input Gate : sigmoid & tanh 함수조합, 새로운 정보의 비율을 정하고 그 값을 업데이트
      - Output Gate : 다음레이어 전달할 값을 결정
    - 게이트들의 조합으로 필요한 정보만 남기고 불필요한 정보를 제거


In [7]:
from ast import Num
# 영화리뷰
import tensorflow as tf
(x_train,y_train),(x_test, y_test) =  tf.keras.datasets.imdb.load_data(num_words=10000)
# 문장의 길이를 맞춰줌
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=200)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,maxlen=200)
# 모델
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=100, input_length=200),
    tf.keras.layers.LSTM(units=128, return_sequences=False),  # 마지막 타임스템프만 반
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
x_train.shape, y_train.shape

((25000, 200), (25000,))

In [9]:
model.fit(x_train, y_train, epochs=3, batch_size=128, validation_split=0.2)
model.evaluate(x_test, y_test)

Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - accuracy: 0.6362 - loss: 0.6063 - val_accuracy: 0.8260 - val_loss: 0.4178
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.8789 - loss: 0.2995 - val_accuracy: 0.8594 - val_loss: 0.3303
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.9275 - loss: 0.2020 - val_accuracy: 0.8644 - val_loss: 0.3305
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8611 - loss: 0.3382


[0.336148738861084, 0.8628000020980835]

- 벡터화된 데이터를 가지고 LSTM모델에 적용
- 동일데이터로 RNN 적용해서 성능 비교

In [10]:
# 하이퍼 파라메터 정의
model_name = 'rnn_classifier_en'
batch_size = 128
num_epochs = 3
vocap_size=10000

In [11]:
class Test():
  def __init__(self) -> None:
    pass
  def __call__(self,x):
    print('호출됨 : ',x)

Test()(100)


호출됨 :  100


In [12]:
# 모델을 클래스
class LSTM_Classifier(tf.keras.Model):
  def __init__(self) -> None:
    super(LSTM_Classifier,self).__init__()
    self.embedding = tf.keras.layers.Embedding(input_dim=vocap_size, output_dim=100, input_length=200)
    self.lstm1 = tf.keras.layers.LSTM(units=128,return_sequences=True) # 모든 은닉레이어의 타임스텝의 출력을 반환
    self.lstm2 = tf.keras.layers.LSTM(units=128) # 시퀀스의 마지막출력만 반환
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.fc1 = tf.keras.layers.Dense(units=10, activation='relu')
    self.fc2 = tf.keras.layers.Dense(units=1, activation='sigmoid')

  def call(self, x):
    x = self.embedding(x)
    x = self.lstm1(x)
    x = self.lstm2(x)
    x = self.dropout(x)
    x = self.fc1(x)
    x = self.fc2(x)
    return x

In [13]:
model = LSTM_Classifier()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [54]:
# callback early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
cp = tf.keras.callbacks.ModelCheckpoint(filepath=f'{model_name}.keras', monitor='val_loss', verbose=1,
                                        save_best_only=True)

In [ ]:
x_train.shape

In [47]:
history = model.fit(x_train, y_train,
                    epochs=num_epochs,
                    batch_size=batch_size,
                    validation_split=0.2,
                    callbacks=[early_stopping, cp])

Epoch 1/3
156/157 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9747 - loss: 0.0755
Epoch 1: val_loss did not improve from 0.36081
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9747 - loss: 0.0756 - val_accuracy: 0.8710 - val_loss: 0.4028
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9811 - loss: 0.0582
Epoch 2: val_loss did not improve from 0.36081
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9810 - loss: 0.0583 - val_accuracy: 0.8708 - val_loss: 0.4975
Epoch 3/3
155/157 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9817 - loss: 0.0534
Epoch 3: val_loss did not improve from 0.36081
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9817 - loss: 0.0535 - val_accuracy: 0.8650 - val_loss: 0.4986


In [50]:
# 모델로드 - > 예측
import tensorflow as tf
model.load_weights('/content/rnn_classifier_en.weights.h5')

In [51]:
model.evaluate(x_val,y_val)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5124 - loss: 1.8707


[1.8802260160446167, 0.5105999708175659]

In [56]:
# 데이터 로드 : 전처리 완료
# https://drive.google.com/file/d/1MaRptad6gke9_Y_t4b0AGStHMcc_JGfz/view?usp=sharing
import gdown
file_id = '1MaRptad6gke9_Y_t4b0AGStHMcc_JGfz'
download_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(download_url, 'train_label.zip', quiet=False)
!unzip /content/train_label.zip

Downloading...
From: https://drive.google.com/uc?id=1MaRptad6gke9_Y_t4b0AGStHMcc_JGfz
To: /content/train_label.zip
100%|██████████| 5.47M/5.47M [00:00<00:00, 34.6MB/s]


Archive:  /content/train_label.zip
replace train_input.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train_input.npy         
  inflating: train_label.npy         


In [57]:
# CNN
# 단어 임베딩 : 단어를 고차원 벡터로 변환   왕, 여왕 서로 가까운 벡터로 표
# 합성곱 : 텍스트의 특징을 추출
# 폴링층
# FC

In [60]:
train_data = np.load('train_input.npy')
train_label = np.load('train_label.npy')
model_name = 'cnn_classifier_en'
batch_size = 128
num_epochs = 3
vocap_size=10000
x_train,x_test,y_train,y_test = train_test_split(train_data, train_label, test_size=0.2, random_state=42)

In [72]:
train_data.shape

(25000, 174)

In [79]:
# 모델선언 컴파일 평가
class CNN_Classifier(tf.keras.Model):
  def __init__(self) -> None:
    super(CNN_Classifier,self).__init__()
    self.embedding = tf.keras.layers.Embedding(input_dim=vocap_size, output_dim=100, input_length=200)
    self.conv1 = tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu')
    self.conv2 = tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu')
    self.conv3 = tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu')
    self.pool1 = tf.keras.layers.MaxPooling1D(pool_size=2)
    self.pool2 = tf.keras.layers.MaxPooling1D(pool_size=2)
    self.pool3 = tf.keras.layers.MaxPooling1D(pool_size=2)
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.fc1 = tf.keras.layers.Dense(units=10, activation='relu')
    self.fc2 = tf.keras.layers.Dense(units=1, activation='sigmoid')
  def call(self,x):
    x = self.embedding(x)
    x = self.pool1( self.conv1(x) )
    x = self.pool2( self.conv2(x) )
    x = self.pool3( self.conv3(x) )
    x = self.dropout(x)  # batch,100,1
    x = tf.keras.layers.Flatten()(x)  # (batch, 1)
    x = self.fc1(x)
    x = self.fc2(x)
    return x

In [80]:
x_train.shape, y_train.shape

((20000, 174), (20000,))

In [81]:
model = CNN_Classifier()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
cp = tf.keras.callbacks.ModelCheckpoint(filepath=f'{model_name}.keras', monitor='val_loss', verbose=1,
                                        save_best_only=True)
history = model.fit(x_train, y_train,
                    epochs=num_epochs,
                    batch_size=batch_size,
                    validation_split=0.2,
                    callbacks=[early_stopping, cp])

Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


113/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5439 - loss: 0.6796
Epoch 1: val_loss improved from inf to 0.52159, saving model to cnn_classifier_en.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.5547 - loss: 0.6740 - val_accuracy: 0.8478 - val_loss: 0.5216
Epoch 2/3
113/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8895 - loss: 0.4450
Epoch 2: val_loss improved from 0.52159 to 0.49421, saving model to cnn_classifier_en.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8900 - loss: 0.4440 - val_accuracy: 0.8600 - val_loss: 0.4942
Epoch 3/3
121/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9350 - loss: 0.3561
Epoch 3: val_loss did not improve from 0.49421
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9350 - loss: 0.3557 - val_accuracy: 0.8602 - val_loss: 0.5047


In [82]:
model.load_weights(f'{model_name}.keras')

In [83]:
model.evaluate(x_test,y_test)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8691 - loss: 0.4663


[0.469328910112381, 0.8668000102043152]

- 영화리뷰 - 이진분류
- 데이터전처리
  - 단어임베딩을위한 문장 벡터화
    - TF-IDF, Word2Vec, CounterVectorizer
  - 모델
    - 머신러닝
      - LogisticRegression ,RandomForest
    - 딥러닝
      - LSTN
      - CNN : Conv1D **

- 자연어 데이터 전치리(기본 step)  
  - Beatifulsoup 을 이용해서 html 태그 제거
  - 소문자
  - 공백을 기준으로 문자열을 리스트화
  - 정규식을 이용해서 알파벳만 추출
  - 다시 문자열로 합치기
  - 임베딩( 3종 세트...)을위한 단어 벡터화

In [84]:
!pip install yfinance newspaper3k nltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 90.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 9.9 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=b5035433f8550d5c60bfa7acf178b7254a1c5383f088b0b9c75542c852525fcc
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=fe36d69461a4fdc91111487790c7a886d770558378f7b869b1c3ae70d43b0724
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc933e4

In [87]:
import yfinance as yf
# 애플 주식데이터
from newspaper import Article
data = yf.download('AAPL',start = '2023-01-01', end = '2024-01-01', interval='1d')

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,130.279999,130.899994,124.169998,125.070000,123.904633,112117500
2023-01-04,126.889999,128.660004,125.080002,126.360001,125.182617,89113600


In [88]:
data.head(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,130.279999,130.899994,124.169998,125.070000,123.904633,112117500
2023-01-04,126.889999,128.660004,125.080002,126.360001,125.182617,89113600
